In [ ]:
!pip install opencv-python
!pip install numpy

import cv2
import numpy as np
import os

In [ ]:
# Define paths to your model files
face_model_path = "/content/opencv_face_detector_uint8.pb"
face_config_path = "/content/opencv_face_detector.pbtxt"
age_model_path = "/content/age_net.caffemodel"
age_config_path = "/content/age_deploy.prototxt"
gender_model_path = "/content/gender_net.caffemodel"
gender_config_path = "/content/gender_deploy.prototxt"

In [ ]:
# Load models
face_net = cv2.dnn.readNet(face_model_path, face_config_path)
age_net = cv2.dnn.readNet(age_model_path, age_config_path)
gender_net = cv2.dnn.readNet(gender_model_path, gender_config_path)

In [ ]:
# Define age and gender lists
age_list = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
gender_list = ['Male', 'Female']

In [ ]:
def detect_and_predict(image):
    # Create a blob from the image
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104, 117, 123))

    # Detect faces
    face_net.setInput(blob)
    detections = face_net.forward()

    # Process each detected face
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:
            # Get face box coordinates
            box = detections[0, 0, i, 3:7] * np.array([image.shape[1], image.shape[0], image.shape[1], image.shape[0]])
            (x, y, x1, y1) = box.astype("int")

            # Extract face ROI
            face = image[y:y1, x:x1]

            # Prepare face for age and gender prediction
            face_blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)

            # Predict gender
            gender_net.setInput(face_blob)
            gender_preds = gender_net.forward()
            gender = gender_list[gender_preds[0].argmax()]

            # Predict age
            age_net.setInput(face_blob)
            age_preds = age_net.forward()
            age = age_list[age_preds[0].argmax()]

            # Draw rectangle and label on the image
            cv2.rectangle(image, (x, y), (x1, y1), (0, 255, 0), 2)
            label = f"{gender}, {age}"
            cv2.putText(image, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    return image


In [ ]:
# Process all images in the directory
image_dir = "."  # Current directory
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_dir, filename)
        image = cv2.imread(image_path)
        if image is not None:
            result = detect_and_predict(image)
            cv2.imwrite(f"output_{filename}", result)
            print(f"Processed {filename}")

print("All images processed. Check the output files.")

Processed kid2.jpg
Processed girl2.jpg
Processed woman1.jpg
Processed girl1.jpg
Processed man1.jpg
Processed man2.jpg
Processed kid1.jpg
All images processed. Check the output files.
